# <b style='color:gold;'> LightLGBM </b>
* Este Jupyter tem como função treinar um `LightLGBM` com os melhores parametros determinados por um `scikit-optimize`

---

Bibliotecas/Módulos

<details>    
<summary>
    <font size="3" color="magenta"><b>Install</b></font>
</summary>
<p>
<ul>
    <li> !pip install lightgbm</li>
    <li> !pip install scikit-optimize </li>

</ul>
</p>

In [1]:
%%capture

!pip install lightgbm scikit-optimize

In [2]:
import logging
import numpy as np
import pandas as pd
import sys
from lightgbm import LGBMClassifier
# from sklearn.externals import joblib
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score, matthews_corrcoef, classification_report, f1_score
from skopt import forest_minimize
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier

Configurações dos logs

In [3]:
logger = logging.getLogger("SVM")
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler(sys.stderr)
formatter = logging.Formatter('%(name)s - %(levelname)s - [+] ------- %(message)s -------') 
handler.setFormatter(formatter)
logger.handlers = [handler]

Cores

In [4]:
RED = "\033[1;31m"
BLUE = "\033[1;34m"
GREEN = "\033[1;32m"
PINK = "\033[1;45m"
HEADER = '\033[95m'
OKBLUE = '\033[94m'
OKCYAN = '\033[96m'
OKGREEN = '\033[92m'
WARNING = '\033[93m'
FAIL = '\033[91m'
ENDC = '\033[0m'
BOLD = '\033[1m'
UNDERLINE = '\033[4m'
MAG = "\033[1;45m"

---

In [25]:
df = pd.read_csv("dados_FEM_MAS_10_voluntarios_s_semanas_10_sinais.csv")

In [26]:
df.sample()

,0,1,2,3,4,5,6,7,8,9,...,1991,1992,1993,1994,1995,1996,1997,1998,1999,label
8841,185,186,-41,19,76,100,-18,-47,45,-74,...,-227,-4,-13,-63,-24,22,-50,47,-38,0


In [27]:
len(df)

17920

In [28]:
len(df.columns)

2001

---

In [29]:
y = df["label"]

In [30]:
df = df.drop(["label"], axis=1)

In [31]:
from sklearn.decomposition import PCA

In [32]:
pca = PCA(n_components=1000)

In [33]:
pca.fit(df)

PCA(n_components=1000)

In [34]:
%%time

pca_samples = pca.transform(df)

CPU times: user 13.9 s, sys: 1.99 s, total: 15.9 s
Wall time: 1.18 s


In [35]:
ps = pd.DataFrame(pca_samples)
ps.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,-897.608954,33.982896,0.004304,-53.764392,-7.444923,-175.937037,-81.112883,118.023585,-80.549260,28.846527,...,-17.781109,19.186073,14.947061,-24.648308,4.880926,-29.219868,10.019273,16.754146,-1.098009,9.293909
1,-899.464432,71.015366,17.799590,-83.367689,2.443263,-142.726944,-45.769773,121.941678,-46.318558,10.323928,...,-20.866480,4.859701,1.610689,-31.722632,14.348585,-16.792268,-4.280823,5.488711,-13.234726,7.566110
2,-913.400121,97.403993,31.376492,-119.715895,5.401714,-103.049353,-31.937180,123.864041,-10.768580,-0.491098,...,-24.774640,-5.919802,-8.265608,-32.504211,9.496170,-2.829562,-12.734673,2.613981,-23.179464,13.789089
3,-910.858586,119.481561,34.003806,-143.894382,-4.435768,-69.941088,-6.300342,115.486438,8.625186,-17.823369,...,-10.673778,-20.483178,-22.121767,-30.985782,5.655899,17.635593,-24.745747,-14.266163,-31.929582,13.762403
4,-885.288426,117.074564,48.482579,-155.122125,-1.253690,-32.133186,9.479085,115.676213,24.557022,-31.582391,...,-1.153732,-18.390297,-25.346063,-28.467946,12.811640,23.237554,-8.771087,-11.912600,-48.808655,7.212890


In [36]:
len(ps)

17920

In [37]:
len(ps.columns)

1000

In [38]:
X = ps

In [39]:
x_treino, x_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=7)

---

### LGBM

Instanciando

In [40]:
%%time

mdl_lgbm = LGBMClassifier(random_state=0, class_weight="balanced", n_jobs=2)
mdl_lgbm.fit(x_treino, y_treino)

CPU times: user 23.3 s, sys: 19.5 ms, total: 23.3 s
Wall time: 11.9 s


LGBMClassifier(class_weight='balanced', n_jobs=2, random_state=0)

Tuning do Lgbm

In [41]:
def tune_lgbm(params):
    print(params)
    lr = params[0]
    max_depth = params[1]
    min_child_samples = params[2]
    subsample = params[3]
    colsample_bytree = params[4]
    n_estimators = params[5]
    
    min_df = params[6]
    ngram_range = (1, params[7])
    
    logger.info('Instanciando')
    mdl_tune = LGBMClassifier(learning_rate=lr, num_leaves=2 ** max_depth, max_depth=max_depth, 
                         min_child_samples=min_child_samples, subsample=subsample,
                         colsample_bytree=colsample_bytree, bagging_freq=1,n_estimators=n_estimators, random_state=0, 
                         class_weight="balanced", n_jobs=2)
    
    logger.info('Treinando')
    mdl_tune.fit(x_treino, y_treino)
    
    logger.info('Predict')
    p = mdl_tune.predict_proba(x_teste)[:, 1]
    categorias = mdl_tune.predict(x_teste)
    
    print('Métricas: \n \n')
    
    logger.info('Roc')
    roc = roc_auc_score(y_teste, p)
    print(OKBLUE, "Roc \n" + str(roc) + "\n")
    
    logger.info('Acurácia')
    acc = accuracy_score(y_teste, categorias)
    print(OKCYAN, "Acc \n" + str(acc) + "\n")
    
    logger.info("Mcc")
    mcc = matthews_corrcoef(y_teste, categorias)
    print(WARNING, "Mcc \n" + str(mcc) + "\n")
    
    logger.info("F1")
    f1 = f1_score(y_teste, categorias, average="weighted")
    print(GREEN, "F1 \n" + str(f1) + "\n")
    
    logger.info('Class Report')
    resultados = classification_report(y_teste, categorias)
    print(ENDC, "Class Report \n" + resultados + "\n")
    
    return -accuracy_score(y_teste, categorias)


In [42]:
space = [(1e-3, 1e-1, 'log-uniform'), # lr
          (1, 10),                    # max_depth
          (1, 20),                    # min_child_samples
          (0.05, 1.),                 # subsample
          (0.05, 1.),                 # colsample_bytree
          (100,1000),                 # n_estimators
          (1,5),                      # min_df
          (1,5)]                      # ngram_range

[0.009944912110647982, 5, 1, 0.4677107511929402, 0.49263223036174764, 272, 3, 1]

In [ ]:
%%time

try:
    res = forest_minimize(tune_lgbm, space, random_state=7, n_random_starts=20, n_calls=50, verbose=1)

except Exception as error:
    logger.warning('Aconteceu algum problema...')
    logger.critical(error)

else:
    logger.info('Ok')
    
finally:
    logger.info('Treino finalizado')

SVM - INFO - [+] ------- Instanciando -------
SVM - INFO - [+] ------- Treinando -------


Iteration No: 1 started. Evaluating function at random point.
[0.002848906260926589, 7, 4, 0.9793117522246534, 0.4828056676618013, 285, 1, 2]


SVM - INFO - [+] ------- Predict -------
SVM - INFO - [+] ------- Roc -------
SVM - INFO - [+] ------- Acurácia -------
SVM - INFO - [+] ------- Mcc -------
SVM - INFO - [+] ------- F1 -------
SVM - INFO - [+] ------- Class Report -------
SVM - INFO - [+] ------- Instanciando -------
SVM - INFO - [+] ------- Treinando -------


Métricas: 
 

 Roc 
0.8601149036184239

 Acc 
0.7321428571428571

 Mcc 
0.5238984690016794

 F1 
0.7179444706055964

 Class Report 
              precision    recall  f1-score   support

           0       0.66      0.96      0.78      1777
           1       0.93      0.51      0.66      1807

    accuracy                           0.73      3584
   macro avg       0.79      0.73      0.72      3584
weighted avg       0.79      0.73      0.72      3584


Iteration No: 1 ended. Evaluation done at random point.
Time taken: 24.5481
Function value obtained: -0.7321
Current minimum: -0.7321
Iteration No: 2 started. Evaluating function at random point.
[0.0013934848991057577, 9, 8, 0.551376555079732, 0.8753613852974182, 491, 5, 1]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


SVM - INFO - [+] ------- Predict -------
SVM - INFO - [+] ------- Roc -------
SVM - INFO - [+] ------- Acurácia -------
SVM - INFO - [+] ------- Mcc -------
SVM - INFO - [+] ------- F1 -------
SVM - INFO - [+] ------- Class Report -------
SVM - INFO - [+] ------- Instanciando -------
SVM - INFO - [+] ------- Treinando -------


...

Métricas: 
 

 Roc 
0.8713628205699152

 Acc 
0.7469308035714286

 Mcc 
0.49764691284797485

 F1 
0.745677143184418

 Class Report 
              precision    recall  f1-score   support

           0       0.78      0.68      0.73      1777
           1       0.72      0.81      0.76      1807

    accuracy                           0.75      3584
   macro avg       0.75      0.75      0.75      3584
weighted avg       0.75      0.75      0.75      3584


Iteration No: 37 ended. Search finished for the next optimal point.
Time taken: 142.6433
Function value obtained: -0.7469
Current minimum: -0.8005
Iteration No: 38 started. Searching for the next optimal point.
[0.004695839573870931, 6, 12, 0.9270776000694007, 0.7564982795530728, 973, 4, 3]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


SVM - INFO - [+] ------- Predict -------
SVM - INFO - [+] ------- Roc -------
SVM - INFO - [+] ------- Acurácia -------
SVM - INFO - [+] ------- Mcc -------
SVM - INFO - [+] ------- F1 -------
SVM - INFO - [+] ------- Class Report -------
SVM - INFO - [+] ------- Instanciando -------


Métricas: 
 

 Roc 
0.8906892753404739

 Acc 
0.771484375

 Mcc 
0.5429583684938859

 F1 
0.7714473778217723

 Class Report 
              precision    recall  f1-score   support

           0       0.77      0.76      0.77      1777
           1       0.77      0.78      0.78      1807

    accuracy                           0.77      3584
   macro avg       0.77      0.77      0.77      3584
weighted avg       0.77      0.77      0.77      3584


Iteration No: 38 ended. Search finished for the next optimal point.
Time taken: 71.5371
Function value obtained: -0.7715
Current minimum: -0.8005
Iteration No: 39 started. Searching for the next optimal point.
[0.0040771182378667925, 10, 4, 0.2800792717418604, 0.44174035942662837, 601, 5, 5]


SVM - INFO - [+] ------- Treinando -------


[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


In [24]:
res.

SyntaxError: invalid syntax (1459325066.py, line 1)

 2 voluntarias
 
 accuracy                           0.79      1536

In [ ]:
res.x